In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import tensorflow as tf
import keras
import sys
import os
import random
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from sklearn.metrics import accuracy_score, classification_report
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Dropout, BatchNormalization, Flatten, AveragePooling2D
from keras import Input
from keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer


In [2]:
main_dir = os.path.dirname(sys.path[0])
family_link = main_dir+'/recognize_family/data/family/'

In [3]:
labels = set(['with_mask','without_mask'])

image_paths = []
image_labels = []

for label in labels:
    for image in list(os.listdir(family_link+label)):
        image_paths = image_paths + [family_link + label + '/' +image]
        image_labels = image_labels + [label]

In [4]:
data = []
for img in image_paths:
    image = cv2.imread(img)
    image = cv2.resize(image,(224,224))
    data.append(image)

In [5]:
X = np.array(data,dtype='float')
y = np.array(image_labels)

In [6]:
lb = LabelBinarizer()
y = lb.fit_transform(y.reshape(-1,1))
y = to_categorical(y)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [8]:
aug = ImageDataGenerator(rotation_range=20,zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,horizontal_flip=True,fill_mode='nearest')

In [9]:
base_model = VGG19(weights='imagenet',include_top=False,input_tensor=Input(shape=(224,224,3)))

head_model = base_model.output
head_model = AveragePooling2D(pool_size=(2,2))(head_model)
head_model = Flatten(name='Flatten')(head_model)
head_model = Dense(128,activation='relu')(head_model)
head_model = Dropout(0.5)(head_model)
head_model = Dense(2,activation='softmax')(head_model)

model = Model(inputs=base_model.input,outputs=head_model)

In [10]:
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

In [11]:
for layer in base_model.layers:
    layer.trainable = False

opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)

model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [12]:
model_run = model.fit(aug.flow(X_train, y_train, batch_size=BS),validation_data=(X_test, y_test),epochs=EPOCHS)

Epoch 1/20
189/189 [==============================] - 2179s 12s/step - loss: 0.8593 - accuracy: 0.8994 - val_loss: 0.0707 - val_accuracy: 0.9848
Epoch 2/20
189/189 [==============================] - 2594s 14s/step - loss: 0.2482 - accuracy: 0.9591 - val_loss: 0.0487 - val_accuracy: 0.9881
Epoch 3/20
189/189 [==============================] - 2654s 14s/step - loss: 0.1476 - accuracy: 0.9712 - val_loss: 0.0393 - val_accuracy: 0.9907
Epoch 4/20
189/189 [==============================] - 2411s 13s/step - loss: 0.1229 - accuracy: 0.9732 - val_loss: 0.0252 - val_accuracy: 0.9934
Epoch 5/20
189/189 [==============================] - 2405s 13s/step - loss: 0.0934 - accuracy: 0.9786 - val_loss: 0.0248 - val_accuracy: 0.9927
Epoch 6/20
189/189 [==============================] - 2022s 11s/step - loss: 0.0803 - accuracy: 0.9810 - val_loss: 0.0218 - val_accuracy: 0.9960
Epoch 7/20
189/189 [==============================] - 2050s 11s/step - loss: 0.0790 - accuracy: 0.9790 - val_loss: 0.0211 - val_ac

In [13]:
loss, acc = model.evaluate(X_test,y_test)

print('loss : ',loss)
print('accuracy : ',acc*100)

48/48 [==============================] - 425s 9s/step - loss: 0.0155 - accuracy: 0.9934
loss :  0.015534164384007454
accuracy :  99.33818578720093


In [14]:
model.save("vgg_mask_detector.model", save_format="h5")